# Part 2: Feature Engineering, Modeling, & Evaluation

## Imports

In [ ]:
import pandas                as pd
import numpy                 as np
import matplotlib.pyplot     as plt
import seaborn               as sns
import xgboost               as xgb
from math                    import sqrt
from sklearn.linear_model    import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics         import r2_score, mean_squared_error, mean_absolute_error 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing   import StandardScaler
from sklearn.ensemble        import RandomForestRegressor
from sklearn.pipeline        import Pipeline
from IPython.core.display    import display, HTML
from IPython.display         import display_html
sns.set(style = "white", palette = "husl")
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

## Table Of Contents


1. [Reading In The Data](#Reading-In-The-Data)
    - [Overview](#Overview)


2. [Feature Engineering](#Feature-Engineering)
    - [Transforming Numeric Data](#Transforming-Numeric-Data)
    - [Creating Segmental Features](#Creating-Segmental-Features)
    

3. [Modeling](#Modeling)
    - [Functions](#Functions)
    - [Linear Regression](#Linear-Regression)
    - [Ridge](#Ridge)
    - [LASSO](#LASSO)
    - [ElasticNetCV](#ElasticNetCV)
    - [Random Forest Reression](#Random-Forest-Regression)
    - [Prediction Dataframes](#Prediction-Dataframes)
    

4. [Model Evaluation](#Model-Evaluation)
    - [Functions](#Evaluation-Functions)
    - [Plotting Predictions](#Plotting-Predictions)
    - [Plotting Residuals](#Plotting-Residuals)

## Reading In The Data

In [ ]:
mri = pd.read_csv("../Data/mri_cleaned.csv")

### Overview

In [ ]:
mri.head()

In [ ]:
# Checking the shape of the data

print(f"The shape of the dataset is: {mri.shape}")

In [ ]:
# Summary of column data types

mri.dtypes.value_counts()

## Feature Engineering

### Transforming Numeric Data

There are only four numeric columns in the data set: `age`, `lvesv`, `lvedv`, `lvef`.  Of the four, only `lvef` does not have any kind of a normal distribution: `age` is close to normally distributed, while `lvesv` and `lvedv` are log-normally distributed.

We cannot do anything to `lvedv` because that is my target variable, but we can take the log of `lvesv` (in this case the natural log).  We also tried squaring `age` but that did not affect the distribution in the way we hoped it would.

In [ ]:
# Take the natural log of `lvesv`.
# We chose to make it it's own column rather
# than overwrite the column.

mri["lvesv_log"] = mri["lvesv"].apply(lambda x: np.log(x))

In [ ]:
print(f"The shape of the dataset is: {mri.shape}")

[Top](#Table-Of-Contents)

### Creating Segmental Features

Because the model attempts to predict the end diastolic volume, we want it to be as accurate as it can be.  As part of that, we will try to use different combinations of features in an attempt to achieve high accuracy.

The data have 34 columns that we wish to engineer: a column measuring scarification and a column measuring ischemia.  Because there are so many of them, we felt the need to experiment with how they are passed into the model.  We are unable to create multiplication interaction columns, because there are zeros.  Instead, we elected to create segmental columns by summing similar columns together: we will compare the model's performance with the originals and with the segmental columns.

We used this image to guide our create of segmental columns:

<img src = "../Images/cardiac-segmentation-for-cardiac-perfusion-defects.jpg" alt = "Cardiac Segmentation" height = 750 width = 750>

Case courtesy of Dr Hamid Chalian, <a href="https://radiopaedia.org/">Radiopaedia.org</a>. From the case <a href="https://radiopaedia.org/cases/47102">rID: 47102</a>

In [ ]:
# Creating columns based off of the red, blue, green, and yellow
# We combined the apex and apical regions because there is only
# one region in the apex.

# New columns for scar tissue

mri["basal_he"]    = mri["ba_he"] + mri["bas_he"] + mri["bis_he"] \
                     + mri["bi_he"] + mri["bil_he"] + mri["bal_he"]
mri["mid_he"]      = mri["ma_he"] + mri["mas_he"] + mri["mis_he"] \
                     + mri["mi_he"] + mri["mil_he"] + mri["mal_he"]
mri["apical_he"]   = mri["aa_he"] + mri["as_he"] + mri["ai_he"] \
                     + mri["al_he"] + mri["apex_he"]

# New columns for ischemia

mri["basal_ischemia"]  = mri["ba_ischemia"] + mri["bas_ischemia"] + mri["bis_ischemia"] \
                         + mri["bi_ischemia"] + mri["bil_ischemia"] + mri["bal_ischemia"]
mri["mid_ischemia"]    = mri["ma_ischemia"] + mri["mas_ischemia"] + mri["mis_ischemia"] \
                         + mri["mi_ischemia"] + mri["mil_ischemia"] + mri["mal_ischemia"]
mri["apical_ischemia"] = mri["aa_ischemia"] + mri["as_ischemia"] + mri["ai_ischemia"] \
                         + mri["al_ischemia"]

print(f"The shape of the dataset is: {mri.shape}")

We intended to create new columns based off the secondary axis (anteroseptal, inferoseptal, etc.) but because of how the apical region is divided we would have counted the apical regions more than once.

[Top](#Table-Of-Contents)

One of our goals was to see what would happen if we used the engineered features to reduce the overall number of reatures being passed into the model.  To accomplish that, we created two dataframes: one with the original set of features (48 including the target) and a second one with the engineered features (21 including the target).


From this point on, `_org` will be used to refer to the original set of features and `_eng` will be use to refer to the engineered features.  The default will be `_org`, so after train-splitting the originial will just be `X_train` or `X_train_ss`.

In [ ]:
# Defining the original dataframe

mri_org = mri.drop(labels = ["lvesv_log", "basal_he", "mid_he",
                             "apical_he", "basal_ischemia",
                             "mid_ischemia", "apical_ischemia"],
                   axis = 1)

# Defining the dataframe with only new features

mri_eng = mri.drop(labels = ['lvesv', 'ba_he', 'bas_he', 'bis_he','bi_he', 
                             'bil_he', 'bal_he', 'ma_he', 'mas_he', 'mis_he', 
                             'mi_he', 'mil_he','mal_he', 'aa_he', 'as_he', 
                             'ai_he', 'al_he', 'apex_he', 'ba_ischemia',
                             'bas_ischemia', 'bis_ischemia', 'bi_ischemia', 
                             'bil_ischemia','bal_ischemia', 'ma_ischemia', 
                             'mas_ischemia', 'mis_ischemia','mi_ischemia', 
                             'mil_ischemia', 'mal_ischemia', 'aa_ischemia', 
                             'as_ischemia', 'ai_ischemia', 'al_ischemia'],
                   axis = 1)

# Checking to make sure the two have different numbers of columns

print(f"The shape of the dataframe with original features is  : {mri_org.shape}")
print(f"The shape of the dataframe with engineered features is: {mri_eng.shape}")

[Top](#Table-Of-Contents)

## Modeling

Before we can start modeling, we have to perform a train-test split.  A train-test split allows us to train our data on one subset of the dataframe and train on another subset.

Since I have two versions of the dataframe, I will have to train-test split on both.

In [ ]:
# Setting up my X and y variables for the original

X = mri_org.drop("lvedv", axis = 1)
y = mri_org["lvedv"]

# Setting up my X and y variables for the new

X_eng = mri_eng.drop("lvedv", axis = 1)
y_eng = mri_eng["lvedv"]

In [ ]:
# Train-test splitting mri_og

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    random_state = 42,
                                                    test_size    = 0.25) 

# Train-test splitting mri_new

X_train_eng, X_test_eng, y_train_eng, y_test_eng = train_test_split(X_eng, 
                                                                    y_eng,
                                                                    random_state = 42,
                                                                    test_size    = 0.25) 

Each model going forward will be fit and evaluated on both sets of train-test splits.

Since I will be running models that need to have the data scaled, I will run both sets of X variables through `StandardScaler`.

In [ ]:
# Instantiating the standard scaler

ss = StandardScaler()

In [ ]:
# Transforming the set with original 
# features.

# Fit-transforming the X_train features

X_train_ss = ss.fit_transform(X_train)

# Transforming my X_test variables

X_test_ss  = ss.transform(X_test)

In [ ]:
# Transforming the sub-set with the
# features I engineered

# Fit-transforming the engineered X_train

X_train_eng_ss = ss.fit_transform(X_train_eng)

# Transforming the engineered X_test features

X_test_eng_ss  = ss.transform(X_test_eng)

### Functions

In [ ]:
# Formula to generate an adjusted r^2 score for
# my model evaluation function in the next cell

# This formula was take from the linear regression lab

def r2_adj(X, y_true, y_predicted):
    r2          = r2_score(y_true, y_predicted)
    numerator   = (1 - r2) * (len(y) - 1)
    denominator = (len(y) - len(X.columns)) - 1
    quotient    = numerator / denominator
    r2_adj      = 1 - quotient
    return r2_adj

The `model_evaluation` function has four metrics:

* r<sup>2</sup> to measure variance in `y` caused by the `X` variables;


* adjusted r<sup>2</sup> to measure the same, but weighted for the number of features;


* root mean squared error (RMSE) to measure the difference between my predicted values and the actual values;


* mean absolute error (MAE) to measure the absolute mean error (this method is less sensitive to outliers).

In [ ]:
def model_evaluation(X, y_true, y_predicted):
    rmse = sqrt(mean_squared_error(y_true, y_predicted))
    mae  = mean_absolute_error(y_true, y_predicted)
    r2   = r2_score(y_true, y_predicted)
    print(f"The root mean squared error is : {rmse}")
    print(f"The mean absolute error is     : {mae}")
    print(f"The r2 score is                : {r2}")
    print(f"The adjusted r2 score is       : {r2_adj(X, y_true, y_predicted)}")

In [ ]:
# This is the same as the cell above, but lacking the adjusted r^2 score
# because after standard scaling the `X` variables the columns are lost.

def model_evaluation_nor2adj(y_true, y_predicted):
    rmse = sqrt(mean_squared_error(y_true, y_predicted))
    mae  = mean_absolute_error(y_true, y_predicted)
    r2   = r2_score(y_true, y_predicted)
    print(f"The root mean squared error is : {rmse}")
    print(f"The mean absolute error is     : {mae}")
    print(f"The r2 score is                : {r2}")

The following code is from user ntg on StackOverflow.  [Here](https://stackoverflow.com/a/44923103) is the source.

In [ ]:
# This allows me to display the dataframes of the linear regression
# coefficients side-by-side to save space.

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table', 'table style="display:inline"'), raw = True)

[Top](#Table-Of-Contents)

### Linear Regression

When thinking about our approach to modeling, we decided to start with the simplest model we could think of which was the linear regression.

#### Original Set

The original set is the dataframe with its original 48 columns.

In [ ]:
# Instantiating the linear regression

lr = LinearRegression()

In [ ]:
# Fitting the linear regression to the original
# (non-engineered) subset

lr.fit(X_train, y_train);

In [ ]:
# Generating predictions from the training set

train_preds = lr.predict(X_train)

# Generating my test predictions

lr_preds = lr.predict(X_test)

In [ ]:
model_evaluation(X_train, y_train, train_preds)

In [ ]:
model_evaluation(X_test, y_test, lr_preds)

Based off of how similar the scores are, it is clear that the model is performing very well.

The model is technically overfit, but it is so minor we decided to ignore the overfit.

In [ ]:
# Creating a dataframe makes looking the coefficients easier

lin_reg_org = pd.DataFrame(data    = lr.coef_,
                           index   = X_test.columns,
                           columns = ["Lin_Reg_Org"])

# Sorting for the head and tail five coefficients

head = lin_reg_org.sort_values(by = "Lin_Reg_Org", ascending = False).head()
tail = lin_reg_org.sort_values(by = "Lin_Reg_Org", ascending = False).tail()

display_side_by_side(head,tail)

Displaying the heads and tails displays showed us the strongest and weakest coefficients as determined during the modeling process.

We found it interesting that sex and smoker status were so strong, but hyperlipidemia was so weak in comparison: hicgh cholesterol is a significant factor for cardiac disease.

[Top](#Table-Of-Contents)

#### Engineered Subset

The engineered subset is the dataframe but with the features we engineered instead of the original 34 segmental features.

In [ ]:
# Instantiating the linear model

lr_2 = LinearRegression()

In [ ]:
# Fitting the linear regression to the subset
# with the features I engineered

lr_2.fit(X_train_eng, y_train_eng);

In [ ]:
# Generating predictions for the training set

train_preds = lr_2.predict(X_train_eng)

# Generating my predictions for the testing set

lr_2_preds  = lr_2.predict(X_test_eng)

In [ ]:
model_evaluation(X_train_eng, y_train_eng, train_preds)

In [ ]:
model_evaluation(X_test_eng, y_test_eng, lr_2_preds)

The scores for the engineered subset are worse than for the original dataset, but they are even more consistant.

This was firm evidence for us that a linear type model likely is the best way to model this data.

In [ ]:
# It is easier to view the coefficients in a df

lin_reg_eng = pd.DataFrame(data  = lr_2.coef_,
                           index = X_test_eng.columns,
                           columns = ["Lin_Reg_Eng"])

In [ ]:
head = lin_reg_eng.sort_values(by = "Lin_Reg_Eng", ascending = False).head()
tail = lin_reg_eng.sort_values(by = "Lin_Reg_Eng", ascending = False).tail()

display_side_by_side(head,tail)

[Top](#Table-Of-Contents)

### Ridge

Ridge regression works by imposing a penalty on the coefficients: it uses the $\ell$<sub>2</sub> regularization which shrinks thems towards 0 and effectively removes features without actually doing so and reduces the overall complexity.


Because the features can be so strongly affected by the regularization it is important to have them all be on the same scale; this was done aboe with `StandardScaler`.

#### Original Set

In [ ]:
# Instantiating the ridge model

ridge = RidgeCV()

In [ ]:
# Fitting the ridge model to the training data

ridge.fit(X_train_ss, y_train);

In [ ]:
# Generating predictions for the training set

train_preds = ridge.predict(X_train_ss)

# Generating Predictions for the testing set

ridge_preds = ridge.predict(X_test_ss)

Becuase the RidgeCV and following models use scaled data, they lose the feature columns and instead use arrays of data.  For this reason, an adjusted r<sup>2</sup> cannot be used.

In [ ]:
model_evaluation_nor2adj(y_train, train_preds)

In [ ]:
model_evaluation_nor2adj(y_test, ridge_preds)

This Ridge model's performance is very similar to the linear model's performance which was surprising to us because we had expected the regularization to improve its performance.

#### Engineered Subset

In [ ]:
# Instantiating the ridge model

ridge_2 = RidgeCV()

In [ ]:
# Fitting the ridge model to the data
# with engineered features.

ridge_2.fit(X_train_eng_ss, y_train_eng);

In [ ]:
# Generating predictions for the trianing data

train_preds = ridge_2.predict(X_train_eng_ss)

# Generating Predictions

ridge_2_preds = ridge_2.predict(X_test_eng_ss)

In [ ]:
model_evaluation_nor2adj(y_train_eng, train_preds)

In [ ]:
model_evaluation_nor2adj(y_test_eng, ridge_2_preds)

[Top](#Table-Of-Contents)

### LASSO

LASSO (Least Absolute Shrinkage and Selection Operator) is conceptually similar to Ridge in that it penalizes coefficients, but it uses $\ell$<sub>2</sub> regularization which actually reduces the coefficients of unimportant features to exactly 0.

LASSO also needs to have the features on the same scale.

#### Original Set

In [ ]:
# Instantiating the LASSO model

lasso = LassoCV()

In [ ]:
# Fitting the lasso to my training data
# without engineered features

lasso.fit(X_train_ss, y_train)

In [ ]:
# Generating predictions for the training data

train_preds = lasso.predict(X_train_ss)

# Generating predictions for the testing data

lasso_preds = lasso.predict(X_test_ss)

In [ ]:
model_evaluation_nor2adj(y_train, train_preds)

In [ ]:
model_evaluation_nor2adj(y_test, lasso_preds)

#### Engineered Subset

In [ ]:
# Instantiating the lasso model

lasso_2 = LassoCV()

In [ ]:
# Fitting the lasso to my training data
# with engineered features

lasso_2.fit(X_train_eng_ss, y_train_eng)

In [ ]:
# Generating predictions for the training data

train_preds = lasso_2.predict(X_train_eng_ss)

# Generating predictions for the testing data

lasso_2_preds = lasso_2.predict(X_test_eng_ss)

In [ ]:
model_evaluation_nor2adj(y_train, train_preds)

In [ ]:
model_evaluation_nor2adj(y_test_eng, lasso_2_preds)

[Top](#Table-Of-Contents)

### ElasticNetCV

Elastic net models are similar to both Ridge _and_ LASSO models: they are a compromise between the $\ell$<sub>1</sub> and$\ell$<sub>2</sub> regularizations.  In practice the model drives down non-significant features but does not remove them entirely.

-----

Like the Ridge and LASSO models, an ElasticNet needs to have the data be on the same scale.

#### Original Set

In [ ]:
# Instantiating the model

enet = ElasticNetCV()

In [ ]:
# Fitting the model to the 
# original data set

enet.fit(X_train_ss, y_train)

In [ ]:
# Generating training predictions

train_preds = enet.predict(X_train_ss)

# Generating testing predictions

enet_preds = enet.predict(X_test_ss)

In [ ]:
model_evaluation_nor2adj(y_train, train_preds)

In [ ]:
model_evaluation_nor2adj(y_test, enet_preds)

#### Engineered Subset

In [ ]:
# Instantiatig the model

enet_2 = ElasticNetCV()

In [ ]:
# Fitting the model to the data
# with engineered features

enet_2.fit(X_train_eng_ss, y_train_eng)

In [ ]:
# Generating training predictions

train_preds = enet_2.predict(X_train_eng_ss)

# Generating testing predictions

enet_2_preds = enet_2.predict(X_test_eng_ss)

In [ ]:
model_evaluation_nor2adj(y_train_eng, train_preds)

In [ ]:
model_evaluation_nor2adj(y_test_eng, enet_2_preds)

[Top](#Table-Of-Contents)

### Random Forest Regression

A random forest regressor works by bagging (bootstrap aggregation) random samples of the original dataframe and uses a metric (in our case MSE) to determine the quality of a split.  When it comes to predicting, it generates the mean of each node.

We chose to use a pipeline to make it easier for us to find optimal hyperparameter values; hyperparameters are parameters defined by the individual running the code.

#### Original Set

In [ ]:
# Instantiating the pipeline

rfr_pipe = Pipeline([("rfr", RandomForestRegressor(random_state = 42))])

In [ ]:
# Setting the values for the gridsearch
# These are the optimized values

params = {"rfr__n_estimators"     : [35],
          "rfr__max_depth"        : [None],
          "rfr__min_samples_split": [10],
          "rfr__min_samples_leaf" : [2,],
          "rfr__n_jobs"           : [6]}

# Instantiating the gridsearch

rfr_gs = GridSearchCV(rfr_pipe, 
                      param_grid = params,
                      cv         = 5)

# Fitting the gridsearch to the training data

rfr_gs.fit(X_train, y_train);

# Getting the best parameters

rfr_gs.best_params_

In [ ]:
# Generating training predictions

train_preds = rfr_gs.predict(X_train)

# Generating testing predictions

rfr_preds = rfr_gs.predict(X_test)

In [ ]:
model_evaluation(X_train, y_train, train_preds)

In [ ]:
model_evaluation(X_test, y_test, rfr_preds)

#### Engineered Subset

In [ ]:
# Setting the values for the gridsearch
# These are the optimized values

params = {"rfr__n_estimators"     : [35],
          "rfr__max_depth"        : [None],
          "rfr__min_samples_split": [2],
          "rfr__min_samples_leaf" : [2],
          "rfr__n_jobs"           : [6]}

# Instantiating the gridsearch

rfr_2_gs = GridSearchCV(rfr_pipe, 
                        param_grid = params,
                        cv         = 5)

# Fitting the gridsearch to the training data

rfr_2_gs.fit(X_train_eng, y_train_eng);

# Getting the best parameters

rfr_2_gs.best_params_

In [ ]:
# Generating training predictions

train_preds = rfr_2_gs.predict(X_train_eng)

# Generating testing predictions

rfr_2_preds = rfr_2_gs.predict(X_test_eng)

In [ ]:
model_evaluation(X_train_eng, y_train_eng, train_preds)

In [ ]:
model_evaluation(X_test_eng, y_test_eng, rfr_2_preds)

### Prediction Dataframes

#### Original Features

In [ ]:
# Creating a dataframe out of the four sets of predictions
# and the y_test true values for comparison

org_model_predictions = pd.DataFrame([np.array(y_test), lr_preds, ridge_preds, 
                                      lasso_preds, enet_preds, rfr_preds])

# Transposing the dataframe so it has five columns

org_model_predictions = org_model_predictions.T

# Renaming the columns for each model

org_model_predictions.rename({0: "Actual", 1: "Linear Reg.", 2: "RidgeCV",
                              3: "LassoCV", 4: "ElasticNetCV", 5: "Random Forest Reg."}, 
                              axis = 1,
                              inplace = True)

# Setting the index to the X_test index
# and sorting from lowest to highest

org_model_predictions = org_model_predictions.set_index(X_test.index).sort_index(ascending = True)

# Saving as a .csv file

org_model_predictions.to_csv("../Data/original_model_predictions.csv")

#### Engineered Features

In [ ]:
# Creating a dataframe out of the four sets of predictions
# and the y_test true values for comparison

eng_model_predictions = pd.DataFrame([np.array(y_test_eng), lr_2_preds, ridge_2_preds, 
                                     lasso_2_preds, enet_2_preds, rfr_2_preds])

# Transposing the dataframe so it has five columns

eng_model_predictions = eng_model_predictions.T

# Renaming the columns for each model

eng_model_predictions.rename({0: "Actual", 1: "Linear Reg.", 2: "RidgeCV",
                              3: "LassoCV", 4: "ElasticNetCV", 5: "Random Forest Reg."}, 
                              axis = 1,
                              inplace = True)

# Setting the index to the X_test index
# and sorting from lowest to highest

eng_model_predictions = eng_model_predictions.set_index(X_test.index).sort_index(ascending = True)

# Saving as a .csv file

eng_model_predictions.to_csv("../Data/engineered_model_predictions.csv")

#### Reading In The Files

In [ ]:
original_features   = pd.read_csv("../Data/original_model_predictions.csv", index_col = 0)
engineered_features = pd.read_csv("../Data/engineered_model_predictions.csv", index_col = 0)

In [ ]:
# Checking that the df looks correct

original_features.head()

In [ ]:
# Checking that the df look correct

engineered_features.head()

## Model Evaluation

### Evaluation Functions

This function simply plots the predictions from all four models together with the true `y` values in one scatter plot.

In [ ]:
def plot_predictions(df, true, title):
    
    # Setting the `facecolor` to white makes
    # it easier to see on a different background.
    plt.figure(figsize   = (18,6),
               facecolor = "white")
    
    # Generating a scatter plot for each set of
    # values, but they'll be on one chart.
    plt.scatter(x = true, y = "Actual", data = df,
                color = "black", label = "Actual")
    plt.scatter(x = true, y = "Linear Reg.", data = df,
                color = "red", label = "Linear Reg.")
    plt.scatter(x = true, y = "RidgeCV", data = df,
                color = "blue", label = "RidgeCV")
    plt.scatter(x = true, y = "LassoCV", data = df,
                color = "green", label = "LassoCV")
    plt.scatter(x = true, y = "ElasticNetCV", data = df,
                color = "orange", label = "ElasticNetCV")
    
    # Setting graph parameters
    plt.title(f"Predictions From The {title}", size = 18)
    plt.xlabel("Predicted", size = 16)
    plt.ylabel("Actual", size = 16)
    plt.xticks(size = 14)
    plt.yticks(size = 14)
    plt.legend(bbox_to_anchor = (1.04,1), loc = "upper left");

In [ ]:
# The same as the plots in my first notebooks,
# but graphing the residuals.

def plot_residuals(df, columns, titles):
    count = 0
    fig   = plt.figure(figsize = (18,8))
    for c, column in enumerate(columns):
        count += 1
        ax    = fig.add_subplot(2, 2, count)
        plt.title(f"{titles[c]}", size = 18)
        sns.residplot(x = "Actual",
                      y = column,
                      data = df)
        plt.xlabel("Predicted", size = 16)
        plt.ylabel("Actual", size = 16)
        plt.xticks(size = 14)
        plt.yticks(size = 14)
    plt.tight_layout();
    plt.show();

### Plotting Predictions

While we had a good idea of how the models were performing ased on the scores calculated by our metrics, we felt that visualizing the predictions of the model would help us have a better idea of where the models were strong and weak.  To do that, we simply plotting the four sets of predictions along with the true values in  a scatter plot.

In [ ]:
plot_predictions(df    = original_features, 
                 true  = "Actual", 
                 title = "Original Features")

The four models performed the best on values <200 given that the points there are overlapping so much.  However for values >200, the predictions follow the actual values but they became much more disperse: the variance increased, but not an extreme amount.

In [ ]:
plot_predictions(df    = engineered_features, 
                 true  = "Actual", 
                 title = "Engineered Features")

The models run on the engineered features were also very dense in the 100 to 200 range, but increasingly underperformed for values >300.  This is because the variance increases as the quantity of high values decreases.

### Plotting Residuals

The residuals are simply the difference between the actual values and the predicted values.  Looking at the residuals allows us to detect if the independence of errors assumption is violated.

In [ ]:
plot_residuals(df      = original_features,
               columns = ["Linear Reg.", "RidgeCV",
                          "LassoCV", "ElasticNetCV"],
               titles  = ["Linear Regression", "RidgeCV",
                          "LassoCV", "ElasticNetCV"])

These residual plots are not at all surprising given the distribution of my predicted values: they are dense for low values and very disperse for high values.  That means that the data is heteroscedastic, which means that it is violating one of the assumptions of linear models: independence of errors; ideally the points would be scattered randomly along the horizontal line.

In [ ]:
plot_residuals(df      = engineered_features,
               columns = ["Linear Reg.", "RidgeCV",
                          "LassoCV", "ElasticNetCV"],
               titles  = ["Linear Regression", "RidgeCV",
                          "LassoCV", "ElasticNetCV"])

These residual plots are not at all surprising given the distribution of my predicted values: they are dense for low values and very disperse for high values.  It is easy to see where the models were underperforming for low valuess and high values. As a result, this data is also heterscedastic.

### Final Model Choice

Model Performance With The Four Metrics With The Original Features:
    
| Metric             | Linear Regression | RidgeCV | LassoCV | ElasticNetCV |
|:-------------------|:-----------------:|:-------:|:-------:|:------------:|
| RMSE               | 17.47             | 17.46   | 17.45   | 18.71        |
| MAE                | 12.90             | 12.93   | 12.93   | 14.38        |
| R<sup>2</sup>      | 0.9237            | 0.9238  | 0.9239  | 0.9125       |
| Adj. r<sup>2</sup> | 0.9231            | -----   | -----   | -----        |


Model Performance With The Four Metrics With The Engineered Features:

| Metric             | Linear Regression | RidgeCV   | LassoCV | ElasticNetCV |
|:-------------------|:-----------------:|:---------:|:-------:|:------------:|
| RMSE               | 23.30             | 23.11     | 23.16   | 24.88        |
| MAE                | 14.59             | 15.23     | 15.29   | 16.69        |
| R<sup>2</sup>      | 0.8666            | 0.8666    | 0.8659  | 0.8454       |
| Adj. R<sup>2</sup> | 0.8662            | -----     | -----   | -----        |
